# Calculate Sunset and Sunrise
Welcome to this ugly notebook! What you find here: Calculates the earliest and latest sunrise and sunset for:
- the current year
- if everything was in wintertime
- if everything was in summertime

Earliest sunrise is not during longest day. There may be a fancy formula to calculate this. We go bruth force. So we calculate the sunrise and sunset for each day in year 2025 and find the corresponding entry.

In [1]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
from astral import Observer
from astral.sun import sun
from astral.location import Location, LocationInfo
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pytz
from tqdm import tqdm
import json
import consts

## Define

In [2]:
USE_GRID = 'hex'

## Import Data

In [3]:
if USE_GRID == 'nuts':
    gdf_grid = gpd.read_file(consts.PATH_MASTERNUTS)
    gdf_grid['geometry'] = gdf_grid['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))
    PATH_TIMEDATA = consts.PATH_TIMEDATA_NUTS
elif USE_GRID == 'hex':
    gdf_grid = gpd.read_file(consts.PATH_HEXAGON)
    PATH_TIMEDATA = consts.PATH_TIMEDATA_HEX
else:
    raise ValueError(f"Unknown grid type {USE_GRID}")

## Helpers

In [4]:
def transform_date_to_winter_summer_time(dt, local_tz):

    datetime_local = dt.astimezone(local_tz)

    datetime_winter = datetime_local.time()
    datetime_summer = datetime_local.time()
    if bool(datetime_local.dst()):
        datetime_winter = (datetime_local - timedelta(hours=1)).time()
    else:
        datetime_summer = (datetime_local + timedelta(hours=1)).time()

    return {
            'utc': dt,
            'local': datetime_local,
            'time_winter': datetime_winter,
            'time_summer': datetime_summer,
        }

def find_earliest_and_latest_in_list(suns_per_day, event, timeperiod):
    return {
            'earliest': min(suns_per_day[event], key=lambda x: x[timeperiod])[timeperiod],
            'earliest_day': min(suns_per_day[event], key=lambda x: x[timeperiod])['local'].date(),
            'latest': max(suns_per_day[event], key=lambda x: x[timeperiod])[timeperiod],
            'latest_day': max(suns_per_day[event], key=lambda x: x[timeperiod])['local'].date()
        }

## Calculate Time of Sunrise and Sunset

In [ ]:
# Get all possible dates from 2025
dates = [datetime(2025, 1, 1) + timedelta(days=x) for x in range(365)]

timedata = []

with tqdm(total=len(gdf_grid)) as pbar:
    for i, row in gdf_grid.iterrows():

        pbar.update()

        location = Observer(row.geometry.centroid.y, row.geometry.centroid.x)

        # Earliest sunrise is not the same as the longest day. We need to calculate
        # the earliest day. We could do that the proper way. Or we could just bruth
        # force it. You know what we are gonna do..?
        suns_per_day = {
            'sunrise': [],
            'sunset': []
        }

        local_tz = pytz.timezone(row.timezone)

        # Add this when testing own timezones
        # x = row.geometry.centroid.x
        # if x < -7.5:
        #     tz = -60
        # elif x < 7.5:
        #     tz = 0
        # elif x < 22.5:
        #     tz = 60
        # else:
        #     tz = 120
        # local_tz = pytz.FixedOffset(tz)

        for d in dates:
            try:
                s = sun(location, date=d)
                suns_per_day['sunrise'].append(transform_date_to_winter_summer_time(s['sunrise'], local_tz))
                suns_per_day['sunset'].append(transform_date_to_winter_summer_time(s['sunset'], local_tz))
            except ValueError:
                pass

        # Now get the earliest sunrise for winter and summer time
        record = {
            'nuts_id': row['NUTS_ID'],
            'name': row['NAME_LATN'] if 'NAME_LATN' in row else 'hex',
            'timezone': str(local_tz),
            'sunrise': {
                'summer': find_earliest_and_latest_in_list(suns_per_day, 'sunrise', 'time_summer'),
                'winter': find_earliest_and_latest_in_list(suns_per_day, 'sunrise', 'time_winter'),
                'current': {
                    'earliest': min(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local'].time(),
                    'earliest_day': min(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local'].date(),
                    'latest': max(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local'].time(),
                    'latest_day': max(suns_per_day['sunrise'], key=lambda x: x['local'].time())['local'].date()
                }
            },
            'sunset': {
                'summer': find_earliest_and_latest_in_list(suns_per_day, 'sunset', 'time_summer'),
                'winter': find_earliest_and_latest_in_list(suns_per_day, 'sunset', 'time_winter'),
                'current': {
                    'earliest': min(suns_per_day['sunset'], key=lambda x: x['local'].time())['local'].time(),
                    'earliest_day': min(suns_per_day['sunset'], key=lambda x: x['local'].time())['local'].date(),
                    'latest': max(suns_per_day['sunset'], key=lambda x: x['local'].time())['local'].time(),
                    'latest_day': max(suns_per_day['sunset'], key=lambda x: x['local'].time())['local'].date()
                }
            }
        }

        timedata.append(record)

# Store
json.dump(timedata, open(PATH_TIMEDATA, 'w', encoding='UTF-8'), ensure_ascii=False, indent=2, default=str)

100%|██████████| 6247/6247 [01:15<00:00, 82.32it/s]


## Calculate amount of light at certain hours

In [ ]:
timedata_hourly = []

def calculate_hours_for_event(event, chosetime, day, local_tz):
    records = []
    # loop time
    for hour in range(4, 14):
        for minute in [0, 30]:
            dt = datetime.combine(day, datetime.min.time()) + timedelta(hours=hour, minutes=minute)
            dt = local_tz.localize(dt)

            # if DST is active, we need to adjust the time
            if dt.dst():
                dt -= timedelta(hours=1)
                
            # Calculate sunrise
            elevation = location.solar_elevation(dt)
            elevation_category = ''
            if elevation < -6:
                elevation_category = 'night'
            elif elevation < 0:
                elevation_category = 'dawn'
            elif elevation < 6:
                elevation_category = 'sunrise'
            else:
                elevation_category = 'day'

            records.append({
                'hour': hour,
                'minute': minute,
                'elevation_category': elevation_category,
                'elevation': round(elevation, 1)
            })    

    return records


with tqdm(total=len(gdf_grid)) as pbar:
    for cell in timedata:

        pbar.update()

        # if cell['nuts_id'] != 3443:
        #     continue

        local_tz = pytz.timezone(cell['timezone'])

        # Get Grid Cell and Location Object
        grid_cell = gdf_grid[gdf_grid.NUTS_ID == cell['nuts_id']].iloc[0]
        x = grid_cell.geometry.centroid.y
        y = grid_cell.geometry.centroid.x  
        location = Location(LocationInfo(None, None, None, x, y))

        record = {
            'nuts_id': cell['nuts_id'],
            'sunrise': {
                'current': {
                    'latest': calculate_hours_for_event('sunrise', 'current', cell['sunrise']['current']['latest_day'], local_tz)
                },
                'winter': {
                    'latest': calculate_hours_for_event('sunrise', 'winter', cell['sunrise']['winter']['latest_day'], local_tz)
                },
                'summer': {
                    'latest': calculate_hours_for_event('sunrise', 'summer', cell['sunrise']['summer']['latest_day'], local_tz)
                }
            },
            'sunset': {
                'current': {
                    'earliest': calculate_hours_for_event('sunset', 'current', cell['sunset']['current']['earliest_day'], local_tz)
                },
                'winter': {
                    'earliest': calculate_hours_for_event('sunset', 'winter', cell['sunset']['winter']['earliest_day'], local_tz)
                },
                'summer': {
                    'earliest': calculate_hours_for_event('sunset', 'summer', cell['sunset']['summer']['earliest_day'], local_tz)
                }
            }
        }

        timedata_hourly.append(record)

# Store
json.dump(timedata_hourly, open(consts.PATH_TIMEDATA_HOURLY_HEX, 'w', encoding='UTF-8'), ensure_ascii=False, indent=2, default=str)

100%|██████████| 6247/6247 [00:12<00:00, 496.09it/s]


## Example

In [7]:

local_tz = pytz.timezone("Europe/Zurich")
dt = datetime(2025, 3, 12, 6, 44, 54, 287448)
dt = local_tz.localize(dt)

x = 47.3845
y = 8.5299

location = LocationInfo(None, None, None, x, y)

l = Location(location)
e = l.solar_elevation(dt)
e
#l.solar_azimuth(dt)

-0.37155263146699724